In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from Direct_Stiffness_Methodd import BeamComponent, BoundaryCondition, BeamSolver

# Define inputs
E = 1000  # Young's Modulus (1000 Pa)
nu = 0.3  # Poisson's Ratio
A = np.pi * 1.0**2  # Cross-sectional area (approx 6.2832 m^2)
Iy = np.pi / 4.0  # Moment of inertia about y-axis (approx 3.1416 m^4)
Iz = np.pi / 4.0  # Moment of inertia about z-axis (approx 3.1416 m^4)
J = np.pi / 2.0  # Polar moment of inertia (approx 6.2832 m^4)

nodes = np.array([
    [0.0, 0.0, 0.0, 0],  # Node 0 at origin
    [30.0, 40.0, 0.0, 1],  # Node 1 at (30m, 40m, 0m)
])

elements = np.array([
    [0, 1],  # Element connecting node 0 to node 1
])

fixed_nodes = {
    0: (0.0, 0.0, 0.0, 0.0, 0.0, 0.0),  # Fixed support at node 0
    1: (None, None, None, None, None, None),  # Free at node 1
}

fx = 3/5 * 1000  # Scaling force to 1000N magnitude
fy = 4/5 * 1000
loads = {
    1: (fx, fy, 0.0, 0.0, 0.0, 0.0),  # Force at node 1
}

# Set up the beam and boundary conditions
beam = BeamComponent(nodes, elements, E, nu, A, Iy, Iz, J)
bc = BoundaryCondition(fixed_nodes)
for node_id, load in loads.items():
    bc.apply_load(node_id, load)

# Run the analysis
solver = BeamSolver(beam, bc)
displacements, reactions = solver.solve()
print("Static Analysis Results:")
solver.display_results(displacements, reactions)
print("Internal Forces per Element:")
for elem_idx, forces in solver.internal_forces.items():
    print(f"Element {elem_idx}: {np.round(forces, 5)}")

eigvals, eigvecs = solver.solve_buckling()
print("\nBuckling Analysis Results:")
print("Critical Load Factors:", np.round(eigvals, 5))
print("First Buckling Mode Shape:", np.round(eigvecs[:, 0], 5))

# Define plotting functions
def plot_internal_forces(beam, internal_forces):
    """Plot internal forces and moments along each element."""
    for elem_idx, forces in internal_forces.items():
        node1_id, node2_id = beam.elements[elem_idx]
        n1_idx = np.where(beam.nodes[:, 3] == node1_id)[0][0]
        n2_idx = np.where(beam.nodes[:, 3] == node2_id)[0][0]
        n1_loc = beam.nodes[n1_idx, :3]
        n2_loc = beam.nodes[n2_idx, :3]
        length = np.linalg.norm(n2_loc - n1_loc)
        x = [0, length]  # Distance along element
        
        fig, axs = plt.subplots(2, 3, figsize=(15, 8))
        titles = ['$F_x$', '$F_y$', '$F_z$', '$M_x$', '$M_y$', '$M_z$']
        indices = [(0, 6), (1, 7), (2, 8), (3, 9), (4, 10), (5, 11)]
        
        for i, (ax, title, idx_pair) in enumerate(zip(axs.flat, titles, indices)):
            ax.plot(x, [forces[idx_pair[0]], forces[idx_pair[1]]], 'b-o')
            ax.set_title(title)
            ax.set_xlabel('Length (m)')
            ax.set_ylabel('Force (N)' if i < 3 else 'Moment (N·m)')
            ax.grid(True)
        
        fig.suptitle(f'Internal Forces and Moments - Element {elem_idx}')
        plt.tight_layout()
        plt.show()

def plot_deformed_structure(beam, displacements, scale=1000):
    """Plot the original and deformed beam structure in 3D."""
    fig = plt.figure(figsize=(12, 9))
    ax = fig.add_subplot(111, projection='3d')
    
    # Original structure
    for elem in beam.elements:
        n1_idx = np.where(beam.nodes[:, 3] == elem[0])[0][0]
        n2_idx = np.where(beam.nodes[:, 3] == elem[1])[0][0]
        x = [beam.nodes[n1_idx, 0], beam.nodes[n2_idx, 0]]
        y = [beam.nodes[n1_idx, 1], beam.nodes[n2_idx, 1]]
        z = [beam.nodes[n1_idx, 2], beam.nodes[n2_idx, 2]]
        ax.plot(x, y, z, 'b--', label='Original' if elem[0] == 0 else "")
    
    # Deformed structure
    deformed_nodes = beam.nodes[:, :3] + displacements[:, :3] * scale
    for elem in beam.elements:
        n1_idx = np.where(beam.nodes[:, 3] == elem[0])[0][0]
        n2_idx = np.where(beam.nodes[:, 3] == elem[1])[0][0]
        x = [deformed_nodes[n1_idx, 0], deformed_nodes[n2_idx, 0]]
        y = [deformed_nodes[n1_idx, 1], deformed_nodes[n2_idx, 1]]
        z = [deformed_nodes[n1_idx, 2], deformed_nodes[n2_idx, 2]]
        ax.plot(x, y, z, 'r-', label='Deformed' if elem[0] == 0 else "")
    
    ax.scatter(beam.nodes[:, 0], beam.nodes[:, 1], beam.nodes[:, 2], c='blue', label='Nodes')
    ax.set_xlabel('X (m)')
    ax.set_ylabel('Y (m)')
    ax.set_zlabel('Z (m)')
    ax.set_title(f'Beam Structure (Scale Factor: {scale})')
    ax.legend()
    plt.show()

def plot_buckling_mode(beam, eigvec, mode_num=0, scale=1000, points=10):
    """Plot the buckling mode shape with interpolation."""
    fig = plt.figure(figsize=(12, 9))
    ax = fig.add_subplot(111, projection='3d')
    
    mode_displacements = eigvec.reshape(beam.nodes.shape[0], 6)[:, :3]
    
    for elem_idx, elem in enumerate(beam.elements):
        n1_idx = np.where(beam.nodes[:, 3] == elem[0])[0][0]
        n2_idx = np.where(beam.nodes[:, 3] == elem[1])[0][0]
        n1_loc = beam.nodes[n1_idx, :3]
        n2_loc = beam.nodes[n2_idx, :3]
        
        # Original structure
        x_ref = [n1_loc[0], n2_loc[0]]
        y_ref = [n1_loc[1], n2_loc[1]]
        z_ref = [n1_loc[2], n2_loc[2]]
        ax.plot(x_ref, y_ref, z_ref, 'b--', label='Original' if elem_idx == 0 else "")
        
        # Interpolated buckled shape
        t = np.linspace(0, 1, points)
        x_buckled = np.linspace(n1_loc[0], n2_loc[0], points)
        y_buckled = np.linspace(n1_loc[1], n2_loc[1], points)
        z_buckled = np.linspace(n1_loc[2], n2_loc[2], points)
        
        disp_n1 = mode_displacements[n1_idx]
        disp_n2 = mode_displacements[n2_idx]
        for i in range(points):
            interp_factor = t[i]
            disp = (1 - interp_factor) * disp_n1 + interp_factor * disp_n2
            x_buckled[i] += scale * disp[0]
            y_buckled[i] += scale * disp[1]
            z_buckled[i] += scale * disp[2]
        
        ax.plot(x_buckled, y_buckled, z_buckled, 'g-', label=f'Mode {mode_num+1}' if elem_idx == 0 else "")
    
    ax.set_xlabel('X (m)')
    ax.set_ylabel('Y (m)')
    ax.set_zlabel('Z (m)')
    ax.set_title(f'Buckling Mode {mode_num+1} (λ = {eigvals[mode_num]:.2f}, Scale = {scale})')
    ax.legend()
    plt.show()

# Generate plots
plot_internal_forces(beam, solver.internal_forces)
plot_deformed_structure(beam, displacements, scale=1000)
plot_buckling_mode(beam, eigvecs[:, 0], mode_num=0, scale=1000, points=10)